# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096957


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:43,  1.54s/it]

Rendering models:  10%|█         | 3/30 [00:06<00:53,  1.97s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:39,  1.51s/it]

Rendering models:  17%|█▋        | 5/30 [00:06<00:29,  1.17s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:14<00:01,  3.24it/s]

Rendering models:  90%|█████████ | 27/30 [00:14<00:00,  3.28it/s]

Rendering models:  97%|█████████▋| 29/30 [00:15<00:00,  3.58it/s]

Rendering models: 100%|██████████| 30/30 [00:15<00:00,  3.06it/s]

Jnursssmith5263                       0.000291
Jorgearcelarreta                      0.000109
ElisabethB                            0.000136
preshuss1                             0.000225
jnarayanbvg                           0.000118
not-logged-in-db23dc032fc427a4e4d9    0.000110
thiagoxxxx                            0.000173
Mickfurry                             3.381837
not-logged-in-7e0d8dc059afd0481ef6    0.000129
not-logged-in-5144334ca348e105b771    0.000378
not-logged-in-5f30b2bb9b7f9d49da41    0.086448
Arlo-M                                0.000742
crush202020                           0.000185
AlexYoung35                           0.000191
Planetme                              0.000272
RaulE                                 0.000160
pangeli5                              0.000244
dinosora                              0.000200
not-logged-in-04ed9ae880f18762cd9e    0.000140
Lavadude                              0.017250
Manx                                  0.000798
Susymuller   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())